In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import re
from youtube_api import YouTubeDataAPI
import csv

In [2]:
#create a search function for a keyword
def searchEpisode(key, idNum):
    
    rawCap = YouTubeTranscriptApi.get_transcript(idNum)
    df = pd.DataFrame(rawCap)
    
    txtList = []
    for line in df['text']:
        txtList.append(word_tokenize(line.lower()))

    df['text_analyzed'] = txtList
    
    key = key.lower()
    char = ',./\|;: '
    searchList = []
    count = 0
    
    fullString = re.sub('[^A-Za-z0-9]+', ' ', key)
    cutString = fullString.split()
    
    for keyword in cutString:
        for sentence, start in zip(df['text_analyzed'], df['start']):
            if all(word in sentence for word in cutString):
            #if (keyword in title) or (keyword in desc):
                count = count + 1
                searchList.append(start)
        df3 = pd.DataFrame()
        for item in searchList:
            line = df[df['start'] == item]
            df3 = df3.append(line)
        return df3

In [3]:
def searchPlaylist(playlistID, key):
    creds = open('creds.csv', 'r').read()
    api_key = creds.split(',')[4]    
    yt = YouTubeDataAPI(api_key)
    
    ids = yt.get_videos_from_playlist_id(playlistID)
    videoIDs = []
    for dc in ids:
        videoIDs.append(dc['video_id'])
    rawCap = YouTubeTranscriptApi.get_transcript(videoIDs)
    df = pd.DataFrame(rawCap)
    
    return df